In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install strsimpy

In [ ]:
%pylab inline
plt.style.use("bmh")

In [ ]:
!pip install cyrtranslit

In [ ]:
!pip install pyod

In [ ]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from strsimpy.levenshtein import Levenshtein
from strsimpy.normalized_levenshtein import NormalizedLevenshtein
from strsimpy.damerau import Damerau
from strsimpy.optimal_string_alignment import OptimalStringAlignment
from strsimpy.jaro_winkler import JaroWinkler
from strsimpy.longest_common_subsequence import LongestCommonSubsequence
from strsimpy.metric_lcs import MetricLCS
from strsimpy.ngram import NGram
from strsimpy.qgram import QGram
from strsimpy import SIFT4
from catboost import CatBoostClassifier, Pool, cv
from tqdm import tqdm
tqdm.pandas()
import seaborn as sns
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from scipy import sparse
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
import pycountry
import re
import cyrtranslit
from pyod.models.copod import COPOD
import networkx as nx

In [ ]:
train = pd.read_csv('/kaggle/input/sibur20-naming-data/train.csv', index_col=0)
test = pd.read_csv('/kaggle/input/sibur20-naming-data/test.csv', index_col=0)
train.shape, test.shape

# word preprocessing

Привести к нижнему регистру, кириллицу в транслит, убрать все лишние символы.

In [ ]:
train["name_1"] = train["name_1"].str.lower()
train["name_2"] = train["name_2"].str.lower()

test["name_1"] = test["name_1"].str.lower()
test["name_2"] = test["name_2"].str.lower()

In [ ]:
train["name_1"] = train["name_1"].progress_apply(lambda r: cyrtranslit.to_latin(r, 'ru'))
train["name_2"] = train["name_2"].progress_apply(lambda r: cyrtranslit.to_latin(r, 'ru'))

test["name_1"] = train["name_1"].progress_apply(lambda r: cyrtranslit.to_latin(r, 'ru'))
test["name_2"] = train["name_2"].progress_apply(lambda r: cyrtranslit.to_latin(r, 'ru'))

In [ ]:
train.replace(re.compile(r"[^\w\s]"), "", inplace=True)
test.replace(re.compile(r"[^\w\s]"), "", inplace=True)

### train graph

Сформировать граф из трейна, где нодами будут слова, а ребрами - связи между соседними словами. Затем удалить из трейна и теста ноды с числом ребер, больших чем порог.

In [ ]:
G = nx.Graph()

def fill_graph(s):
    last_word = ''
    for i, word in enumerate(s.split()):
        G.add_node(word)
        if i > 0:
            G.add_edge(word, last_word)
        last_word = word
        
train['name_1'].progress_apply(lambda r: fill_graph(r))
train['name_2'].progress_apply(lambda r: fill_graph(r))

G.number_of_edges(), G.number_of_nodes()

### clear words with hight graph indexes

In [ ]:
GRAPH_THRESHOLD = 50

def graph_index(s):
    counter = 0
    for word in s.split():
        try:
            counter += len(list(G.adj[word]))
        except KeyError:
            pass
    return counter

def graph_filter(s):
    result = ''
    for word in s.split():
        if graph_index(word) < GRAPH_THRESHOLD:
            result += word + ' '
    return result.rstrip()

In [ ]:
train['name_1'] = train['name_1'].progress_apply(lambda r: graph_filter(r))
train['name_2'] = train['name_2'].progress_apply(lambda r: graph_filter(r))
test['name_1'] = test['name_1'].progress_apply(lambda r: graph_filter(r))
test['name_2'] = test['name_2'].progress_apply(lambda r: graph_filter(r))

После удаления частых слов часть имен обратилась в ''. Оценим число, удалим их из трейна.

In [ ]:
len(test[(test['name_1'] == '') | (test['name_2'] == '')])

In [ ]:
len(train[(train['name_1'] == '') | (train['name_2'] == '')])

In [ ]:
train[((train['name_1'] == '') | (train['name_2'] == '')) &
      (train['is_duplicate'] == 1)]

In [ ]:
train = train[(train['name_1'] != '') & (train['name_2'] != '')]
train.shape

# ---------------

In [ ]:
#legal_entities = ['ООО', 'ОАО', 'ЗАО', 'ПАО', 'ОДО']

#for entity in tqdm(legal_entities):
#    train.replace(re.compile(f"\W*{entity}\W*"), "", inplace=True)
#    test.replace(re.compile(f"\W*{entity}\W*"), "", inplace=True)

In [ ]:
#legal_entities = ["ltd\.", "co\.", "inc\.", "b\.v\.", "s\.c\.r\.l\.", "gmbh", "pvt\."]

#for entity in tqdm(legal_entities):
#    train.replace(re.compile(f"\W*{entity}\W*"), "", inplace=True)
#    test.replace(re.compile(f"\W*{entity}\W*"), "", inplace=True)

In [ ]:
# ухудшает скор
#shit_words = ['sa', 's a', 'de', 'cv', 'gmb h', 'g mbh', 'llc', 's pa', 'sp a', 'spa', 'ag', 'rl', 's']

#for shit_word in tqdm(shit_words):
#    train.replace(re.compile('\s+{}'.format(shit_word)), "", inplace=True)
#    test.replace(re.compile(f"\s+{shit_word}\s*"), "", inplace=True)

In [ ]:
#countries = [country.name.lower() for country in pycountry.countries]

#for country in tqdm(countries):
#    train.replace(re.compile(f"\s+{entity}\s*"), "", inplace=True)
#    test.replace(re.compile(f"\s+{entity}\s*"), "", inplace=True)

In [ ]:
#print('Dropped', len(train[(train['name_1'] == '') | (train['name_2'] == '')]), 'samples.')

In [ ]:
#train = train[(train['name_1'] != '') & (train['name_2'] != '')]

In [ ]:
#train[train.is_duplicate==1].sample(15)

In [ ]:
#train['name_1'].isna().sum(), train['name_2'].isna().sum(), test['name_1'].isna().sum(), test['name_2'].isna().sum()

# Feature generation

БОЛЬШЕ ФИЧЕЙ ДЛЯ БОГА ФИЧЕЙ!

In [ ]:
levenshtein = Levenshtein()
train["levenstein"] = train.progress_apply(lambda r: levenshtein.distance(r.name_1, r.name_2), axis=1)
test["levenstein"] = test.progress_apply(lambda r: levenshtein.distance(r.name_1, r.name_2), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['levenstein'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['levenstein'], label='zeros')
plt.title('Levenstein')
plt.legend();

In [ ]:
normalized_levenshtein = NormalizedLevenshtein()

train["norm_levenstein"] = train.progress_apply(lambda r: normalized_levenshtein.distance(r.name_1, r.name_2),
                                                axis=1)
test["norm_levenstein"] = test.progress_apply(lambda r: normalized_levenshtein.distance(r.name_1, r.name_2),
                                              axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['norm_levenstein'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['norm_levenstein'], label='zeros')
plt.title('Norm levenstein')
plt.legend();

In [ ]:
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    try:
        res = float(len(c)) / (len(a) + len(b) - len(c))
    except ZeroDivisionError:
        res = 0
    return res

In [ ]:
train["jaccard"] = train.progress_apply(lambda r: get_jaccard_sim(r.name_1, r.name_2), axis=1)
test["jaccard"] = test.progress_apply(lambda r: get_jaccard_sim(r.name_1, r.name_2), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['jaccard'], label='ones', kde=False)
sns.distplot(train[train['is_duplicate'] == 0]['jaccard'], label='zeros', kde=False)
plt.title('Jaccard distance')
plt.legend();

In [ ]:
vectorizer = CountVectorizer(analyzer='char', ngram_range=(1,5))
def ngramm_distance(str_1, str_2):
    vectorizer.fit([str_1 + ' ' + str_2])
    return np.absolute(vectorizer.transform([str_1]) - vectorizer.transform([str_2])).sum()

In [ ]:
train["ngramms"] = train.progress_apply(lambda r: ngramm_distance(r.name_1, r.name_2), axis=1)
test["ngramms"] = test.progress_apply(lambda r: ngramm_distance(r.name_1, r.name_2), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['ngramms'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['ngramms'], label='zeros')
plt.title('Ngramm distance')
plt.legend();

In [ ]:
def calc_1_in_2(str_1, str_2):
    if str_1 == '':
        return 0
    word = str_1.split()[0]
    return int(word in str_2)

In [ ]:
train["1_in_2"] = train.progress_apply(lambda r: calc_1_in_2(r.name_1, r.name_2), axis=1)
test["1_in_2"] = test.progress_apply(lambda r: calc_1_in_2(r.name_1, r.name_2), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 0]['1_in_2'], label='zeros', kde=False)
plt.title('1st word from name_1 in name_2')
plt.legend();

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['1_in_2'], label='ones', kde=False)
plt.title('1st word from name_1 in name_2')
plt.legend();

In [ ]:
train["2_in_1"] = train.progress_apply(lambda r: calc_1_in_2(r.name_2, r.name_1), axis=1)
test["2_in_1"] = test.progress_apply(lambda r: calc_1_in_2(r.name_2, r.name_1), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['2_in_1'], label='ones', kde=False)
plt.title('1st word from name_2 in name_1')
plt.legend();

In [ ]:
sns.distplot(train[train['is_duplicate'] == 0]['2_in_1'], label='zeros', kde=False)
plt.title('1st word from name_2 in name_1')
plt.legend();

In [ ]:
train['name_1_joined'] = train['name_1'].str.replace(' ', '')
train['name_2_joined'] = train['name_2'].str.replace(' ', '')

test['name_1_joined'] = test['name_1'].str.replace(' ', '')
test['name_2_joined'] = test['name_2'].str.replace(' ', '')

In [ ]:
train["jaccard_joined"] = train.progress_apply(lambda r: get_jaccard_sim(r.name_1_joined, r.name_2_joined), axis=1)
test["jaccard_joined"] = test.progress_apply(lambda r: get_jaccard_sim(r.name_1_joined, r.name_2_joined), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['jaccard_joined'], label='ones', kde=False)
plt.title('Jaccard distance on joined strings')
plt.legend();

In [ ]:
sns.distplot(train[train['is_duplicate'] == 0]['jaccard_joined'], label='zeros', kde=False)
plt.title('Jaccard distance on joined strings')
plt.legend();

In [ ]:
levenshtein = Levenshtein()
train["levenstein_joined"] = train.progress_apply(lambda r: levenshtein.distance(r.name_1_joined, r.name_2_joined), axis=1)
test["levenstein_joined"] = test.progress_apply(lambda r: levenshtein.distance(r.name_1_joined, r.name_2_joined), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['levenstein_joined'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['levenstein_joined'], label='zeros')
plt.title('Levenstein distance on joined strings')
plt.legend();

In [ ]:
normalized_levenshtein = NormalizedLevenshtein()

train["norm_levenstein_joined"] = train.progress_apply(lambda r: normalized_levenshtein.distance(r.name_1_joined, r.name_2_joined),
                                                axis=1)
test["norm_levenstein_joined"] = test.progress_apply(lambda r: normalized_levenshtein.distance(r.name_1_joined, r.name_2_joined),
                                              axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['norm_levenstein_joined'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['norm_levenstein_joined'], label='zeros')
plt.title('Norm levenstein distance on joined strings')
plt.legend();

In [ ]:
train["num_chars"] = train.progress_apply(lambda r: np.absolute(len(r.name_1_joined) - len(r.name_2_joined)), axis=1)
test["num_chars"] = test.progress_apply(lambda r: np.absolute(len(r.name_1_joined) - len(r.name_2_joined)), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['num_chars'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['num_chars'], label='zeros')
plt.title('Num chars difference')
plt.legend();

In [ ]:
train["num_words"] = train.progress_apply(lambda r: np.absolute(len(r.name_1_joined.split()) - len(r.name_2_joined.split())), axis=1)
test["num_words"] = test.progress_apply(lambda r: np.absolute(len(r.name_1_joined.split()) - len(r.name_2_joined.split())), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['num_words'], label='ones', kde=False)
plt.title('Num words difference')
plt.legend();

In [ ]:
sns.distplot(train[train['is_duplicate'] == 0]['num_words'], label='zeros', kde=False)
plt.title('Num words difference')
plt.legend();

In [ ]:
def _subseq_count(str_1, str_2):
    counter = 0
    subs = []
    str = ''
    for char in str_1:
        str += char
        subs.append(str)
    for s in subs:
        if s in str_2:
            counter += len(s)
    return counter

def subseq_count(str_1, str_2):
    count = 0
    for i, s in enumerate(str_1):
        count += _subseq_count(str_1[i:], str_2)
    return count

In [ ]:
train["subseq_rate"] = train.progress_apply(lambda r: _subseq_count(r.name_1_joined, r.name_2_joined) + _subseq_count(r.name_2_joined, r.name_1_joined), axis=1)
test["subseq_rate"] = test.progress_apply(lambda r: _subseq_count(r.name_1_joined, r.name_2_joined) + _subseq_count(r.name_2_joined, r.name_1_joined), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['subseq_rate'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['subseq_rate'], label='zeros')
plt.title('Subseq rate')
plt.legend();

In [ ]:
def word_compare(str_1, str_2):
    count = 0
    words = str_1.split()
    for word in words:
        if word in str_2:
            count += 1
    return count

In [ ]:
train["word_compare"] = train.progress_apply(lambda r: word_compare(r.name_1, r.name_2), axis=1)
test["word_compare"] = test.progress_apply(lambda r: word_compare(r.name_1, r.name_2), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['word_compare'], label='ones', kde=False)
plt.title('Num common words')
plt.legend();

In [ ]:
sns.distplot(train[train['is_duplicate'] == 0]['word_compare'], label='zeros', kde=False)
plt.title('Num common words')
plt.legend();

### word indexes

In [ ]:
train["sum_word_index"] = train.progress_apply(lambda r: graph_index(r.name_1 + ' ' + r.name_2), axis=1)
test["sum_word_index"] = test.progress_apply(lambda r: graph_index(r.name_1 + ' ' + r.name_2), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['sum_word_index'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['sum_word_index'], label='zeros')
plt.title('sum_word_index')
plt.legend();

### Demarou

In [ ]:
damerau = Damerau()
train["damerau"] = train.progress_apply(lambda r: damerau.distance(r.name_1, r.name_2), axis=1)
test["damerau"] = test.progress_apply(lambda r: damerau.distance(r.name_1, r.name_2), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['damerau'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['damerau'], label='zeros')
plt.title('damerau')
plt.legend();

### Optimal String Alignment

In [ ]:
optimal_string_alignment = OptimalStringAlignment()
train["optimal_string_alignment"] = train.progress_apply(lambda r: optimal_string_alignment.distance(r.name_1, r.name_2), axis=1)
test["optimal_string_alignment"] = test.progress_apply(lambda r: optimal_string_alignment.distance(r.name_1, r.name_2), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['optimal_string_alignment'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['optimal_string_alignment'], label='zeros')
plt.title('optimal_string_alignment')
plt.legend();

### Jaro-Winkler

In [ ]:
jarowinkler = JaroWinkler()
train["jarowinkler"] = train.progress_apply(lambda r: jarowinkler.similarity(r.name_1, r.name_2), axis=1)
test["jarowinkler"] = test.progress_apply(lambda r: jarowinkler.similarity(r.name_1, r.name_2), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['jarowinkler'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['jarowinkler'], label='zeros')
plt.title('jarowinkler')
plt.legend();

### Longest Common Subsequence

In [ ]:
lcs = LongestCommonSubsequence()
train["lcs_length"] = train.progress_apply(lambda r: lcs.length(r.name_1, r.name_2), axis=1)
test["lcs_length"] = test.progress_apply(lambda r: lcs.length(r.name_1, r.name_2), axis=1)
train["lcs_distance"] = train.progress_apply(lambda r: lcs.distance(r.name_1, r.name_2), axis=1)
test["lcs_distance"] = test.progress_apply(lambda r: lcs.distance(r.name_1, r.name_2), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['lcs_length'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['lcs_length'], label='zeros')
plt.title('lcs_length')
plt.legend();

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['lcs_distance'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['lcs_distance'], label='zeros')
plt.title('lcs_distance')
plt.legend();

### MetricLCS

In [ ]:
metric_lcs = MetricLCS()
train["metric_lcs"] = train.progress_apply(lambda r: metric_lcs.distance(r.name_1, r.name_2), axis=1)
test["metric_lcs"] = test.progress_apply(lambda r: metric_lcs.distance(r.name_1, r.name_2), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['metric_lcs'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['metric_lcs'], label='zeros')
plt.title('metric_lcs')
plt.legend();

### twogram

In [ ]:
twogram = NGram(2)
train["twogram"] = train.progress_apply(lambda r: twogram.distance(r.name_1, r.name_2), axis=1)
test["twogram"] = test.progress_apply(lambda r: twogram.distance(r.name_1, r.name_2), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['twogram'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['twogram'], label='zeros')
plt.title('twogram')
plt.legend();

### Qgram

In [ ]:
qgram = QGram(2)
train["qgram"] = train.progress_apply(lambda r: qgram.distance(r.name_1, r.name_2), axis=1)
test["qgram"] = test.progress_apply(lambda r: qgram.distance(r.name_1, r.name_2), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['qgram'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['qgram'], label='zeros')
plt.title('qgram')
plt.legend();

### SIFT4

In [ ]:
sift = SIFT4()
train["sift"] = train.progress_apply(lambda r: sift.distance(r.name_1, r.name_2), axis=1)
test["sift"] = test.progress_apply(lambda r: sift.distance(r.name_1, r.name_2), axis=1)

In [ ]:
sns.distplot(train[train['is_duplicate'] == 1]['sift'], label='ones')
sns.distplot(train[train['is_duplicate'] == 0]['sift'], label='zeros')
plt.title('sift')
plt.legend();

# Validation scheme

In [ ]:
#compare 4 chars from name_1 and name_2
srez = 4
train['4_str'] = train['name_1'].str[:srez]
mask = train[train['name_1'].str[:srez] == train['name_2'].str[:srez]].copy()

#dict for unique values
dd = dict(zip(train['4_str'].unique(), np.arange(len(train['4_str'].unique()))))
train['4_str'] = train['4_str'].map(dd)
#for substr in train['4_str'].value_counts():
    

In [ ]:
FOLDS = 5

# array of fold categories
cats = [[] for x in range(FOLDS)]
# array of fold lengths
fold_length = [0] * FOLDS

for x in train['4_str'].unique():
    # find fold with minimum length
    min_fold = np.argmin(fold_length)
    num_samples = train[train['4_str'] == x]['name_1'].count()
    
    cats[min_fold].append(x)
    fold_length[min_fold] += num_samples

print(f'Split complete! {FOLDS} folds in {fold_length} sizes.')

In [ ]:
train_cats = cats[0] + cats[1] + cats[2] + cats[3]
val_cats = cats[4]

In [ ]:
train_splitted = train[train['4_str'].isin(train_cats)]
valid_splitted = train[train['4_str'].isin(val_cats)]

# Pseudo-labeling

In [ ]:
def preprocess(X_train_splitted, X_train, X_test, scaler, vectorizer):
    X_train_splitted[:, 2:] = scaler.fit_transform(X_train_splitted[:, 2:])
    X_train[:, 2:] = scaler.transform(X_train[:, 2:])
    X_test[:, 2:] = scaler.transform(X_test[:, 2:])
    
    vectorizer.fit(X_train_splitted[:, 0] + ' ' + X_train_splitted[:, 1])
    X_train_splitted_ngramms = np.absolute(vectorizer.transform(X_train_splitted[:, 0]) - vectorizer.transform(X_train_splitted[:, 1]))
    X_train_ngramms = np.absolute(vectorizer.transform(X_train[:, 0]) - vectorizer.transform(X_train[:, 1]))
    X_test_ngramms = np.absolute(vectorizer.transform(X_test[:, 0]) - vectorizer.transform(X_test[:, 1]))
    
    X_train_splitted = sparse.csr_matrix(X_train_splitted[:, 2:].astype(np.float))
    X_train = sparse.csr_matrix(X_train[:, 2:].astype(np.float))
    X_test = sparse.csr_matrix(X_test[:, 2:].astype(np.float))
    
    X_train_splitted = sparse.hstack([X_train_splitted_ngramms, X_train_splitted])
    X_train = sparse.hstack([X_train_ngramms, X_train])
    X_test = sparse.hstack([X_test_ngramms, X_test])
    
    return X_train_splitted, X_train, X_test

In [ ]:
exclude_columns = ['name_1', 'name_2', 'name_1_joined', 'name_2_joined', '4_str']
columns = train.drop(exclude_columns + ['is_duplicate'], axis=1).columns
columns_lr = ['name_1', 'name_2'] + list(columns)

vectorizer = CountVectorizer(analyzer='char', ngram_range=(1,5), max_features=5000)
scaler = StandardScaler()

columns

### lr

In [ ]:
# Тренируем на разделенной части тренировочного датасета, генерируем лейбл на всем.
# Скалер и энкодер тоже тренируем на разделенной части датасета.
X_train_splitted = train_splitted[columns_lr].values
y_train_splitted = train_splitted['is_duplicate'].values

X_test = test[columns_lr].values
X_train = train[columns_lr].values

X_train_splitted, X_train, X_test = preprocess(X_train_splitted, X_train, X_test, scaler, vectorizer)

lr = LogisticRegression(random_state=42, verbose=True, class_weight='balanced', C=0.01,
                        n_jobs=-1).fit(X_train_splitted, y_train_splitted)

train['lr_labels'] = lr.predict_proba(X_train)[:, 1]
test['lr_labels'] = lr.predict_proba(X_test)[:, 1]

In [ ]:
plot_data = train[train['4_str'].isin(val_cats)]
sns.distplot(plot_data[plot_data['is_duplicate'] == 1]['lr_labels'], label='ones')
sns.distplot(plot_data[plot_data['is_duplicate'] == 0]['lr_labels'], label='zeros')
plt.title('lr_labels')
plt.legend();

### kmeans

In [ ]:
clustering = KMeans(n_clusters=20, random_state=42).fit(X_train_splitted)
train['kmeans_labels'] = clustering.predict(X_train)
test['kmeans_labels'] = clustering.predict(X_test)

In [ ]:
sns.countplot(data=train[train['4_str'].isin(val_cats)], x='kmeans_labels', hue='is_duplicate')
plt.title('kmeans_labels')
plt.legend();

In [ ]:
enc = OneHotEncoder()
OHE_train = enc.fit_transform(train['kmeans_labels'].values.reshape(-1, 1))
OHE_test = enc.transform(test['kmeans_labels'].values.reshape(-1, 1))
train = train.drop('kmeans_labels', axis=1)
test = test.drop('kmeans_labels', axis=1)
train = pd.concat([train, pd.DataFrame(OHE_train.toarray(), index=train.index)], axis=1)
test = pd.concat([test, pd.DataFrame(OHE_test.toarray(), index=test.index)], axis=1)

### COPOD

In [ ]:
clf = COPOD()
clf.fit(train_splitted[columns].values)
train['COPOD'] = clf.predict(train[columns].values)
test['COPOD'] = clf.predict(test[columns].values)
sns.countplot(data=train[train['4_str'].isin(val_cats)], x='COPOD', hue='is_duplicate')
plt.title('copod labels')
plt.legend();

# Catboost

In [ ]:
exclude_columns = ['name_1', 'name_2', 'name_1_joined', 'name_2_joined', '4_str', 'lr_labels']
columns = train.drop(exclude_columns + ['is_duplicate'], axis=1).columns

In [ ]:
train_splitted = train[train['4_str'].isin(train_cats)]
valid_splitted = train[train['4_str'].isin(val_cats)]

X_train = train_splitted[columns].values
y_train = train_splitted['is_duplicate'].values
train_data = Pool(X_train, y_train)
X_train.shape, y_train.shape

In [ ]:
X_valid = valid_splitted[columns].values
y_valid = valid_splitted['is_duplicate'].values
valid_data = Pool(X_valid, y_valid)

In [ ]:
params = {"iterations": 1000,
          "depth": 2,
          "loss_function": "CrossEntropy",
          "verbose": False,
          "eval_metric": "F1",
          "random_seed": 42,
          "learning_rate": 0.8,
#          "auto_class_weights": 'Balanced',
          "use_best_model": True,
          "l2_leaf_reg": 100
          }

model = CatBoostClassifier(**params)
model.fit(train_data, plot=True, eval_set=valid_data)

In [ ]:
pd.DataFrame(model.get_feature_importance(), index=columns).sort_values(by=0, ascending=False).head()

In [ ]:
preds = model.predict(X_valid)
f1_score(y_valid, preds)

In [ ]:
preds = model.predict_proba(X_valid)[:, 1]
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.92]
metrics = []
for thr in thresholds:
    labels = (preds > thr).astype(int)
    #print('-' * 10, 'THRESHOLD =', thr, '-' * 10)
    #print(classification_report(y_valid, labels))
    #print()
    metric = f1_score(y_valid, labels)
    metrics.append(metric)
plt.plot(metrics)

In [ ]:
#X_train = train[columns].values
#y_train = train['is_duplicate'].values
#train_data = Pool(X_train, y_train)

In [ ]:
#params['use_best_model'] = False
#params['iterations'] = 200
#model = CatBoostClassifier(**params)
#model.fit(train_data, plot=True)

In [ ]:
X_test = test[columns].values

In [ ]:
def submit(preds, threshold=0.5, filename='submit.csv', dataset=test):
    labels = (preds > threshold).astype(int)
    result = pd.DataFrame({'pair_id': dataset.index,
                           'is_duplicate': labels})
    print(f'Число положительных классов для threshold={threshold}: {result["is_duplicate"].sum()} / {result["is_duplicate"].mean():.2%}')
    result.to_csv(filename, index=False)
    print('Done!')
    return result

In [ ]:
preds = model.predict_proba(X_test)[:, 1]
results = []
for thr in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    filename = f'submit_{thr}.csv'
    res = submit(preds, threshold=thr, filename=filename, dataset=test)
    results.append(res)

# Ручная валидация

Если ни одна пара даже близко не похожа, можно не пытаться сабмитить - скор будет ниже плинтуса

In [ ]:
idx = results[-1][results[-1]['is_duplicate'] == 1].index
test.loc[idx, ['name_1', 'name_2']].sample(25)